In [52]:
import datetime
import json
import os

In [1]:
import pandas as pd
import numpy as np

In [13]:
pth_station = "../processed_data/metro-bike-stations.csv"

In [14]:
df_stt = pd.read_csv(pth_station)

In [15]:
df_stt

,station,station_name,go_live_date,region,status,lan,lon
0,3005,7th & Flower,7/7/2016,DTLA,Active,34.048500,-118.258537
1,3006,Olive & 8th,7/7/2016,DTLA,Active,34.045540,-118.256668
2,3007,5th & Grand,7/7/2016,DTLA,Active,34.050480,-118.254593
3,3008,Figueroa & 9th,7/7/2016,DTLA,Active,34.046612,-118.262733
4,3010,11th & Maple,7/10/2016,DTLA,Active,34.037048,-118.254868
...,...,...,...,...,...,...,...
281,4498,Vineland & Oxnard,4/14/2020,North Hollywood,Active,34.179890,-118.370422
282,4524,Wilshire & Normandie,4/17/2020,DTLA,Active,34.061630,-118.301193
283,4523,Western & 3rd,4/22/2020,DTLA,Active,34.068748,-118.309280
284,4502,North Hollywood Plaza,4/23/2020,North Hollywood,Active,34.165520,-118.375153


In [16]:
df_get = df_stt[['station', 'go_live_date', 'region']]

In [17]:
datetime.datetime.strptime('7/7/2016', '%m/%d/%Y')

datetime.datetime(2016, 7, 7, 0, 0)

In [18]:
df_get['go_live_date'] = df_get['go_live_date'].map(lambda t: datetime.datetime.strptime(t, '%m/%d/%Y'))

D:\software\Anaconda3\envs\mecsc\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
df_get

,station,go_live_date,region
0,3005,2016-07-07,DTLA
1,3006,2016-07-07,DTLA
2,3007,2016-07-07,DTLA
3,3008,2016-07-07,DTLA
4,3010,2016-07-10,DTLA
...,...,...,...
281,4498,2020-04-14,North Hollywood
282,4524,2020-04-17,DTLA
283,4523,2020-04-22,DTLA
284,4502,2020-04-23,North Hollywood


In [29]:
stt_count = {area: 0 for area in df_get.region.unique()}
stt_count

{'DTLA': 0,
 'Port of LA': 0,
 'Pasadena': 0,
 'Westside': 0,
 'North Hollywood': 0}

In [23]:
quarter_end_date = {
    '2016-q3': '9/30/2016',
    '2016-q4': '12/31/2016',
    '2017-q1': '3/31/2017',
    '2017-q2': '06/30/2017',
    '2017-q3': '9/30/2017',
    '2017-q4': '12/31/2017',
    '2018-q1': '03/31/2018',
    '2018-q2': '06/30/2018',
    '2018-q3': '09/30/2018',
    '2018-q4': '12/31/2018',
    '2019-q1': '03/31/2019',
    '2019-q2': '06/30/2019',
    '2019-q3': '09/30/2019',
    '2019-q4': '12/31/2019',
    '2020-q1': '3/31/2020',
    '2020-q2': '6/30/2020'
}

In [25]:
for k in quarter_end_date:
    quarter_end_date[k] = datetime.datetime.strptime(quarter_end_date[k], '%m/%d/%Y')

In [26]:
quarter_end_date

{'2016-q3': datetime.datetime(2016, 9, 30, 0, 0),
 '2016-q4': datetime.datetime(2016, 12, 31, 0, 0),
 '2017-q1': datetime.datetime(2017, 3, 31, 0, 0),
 '2017-q2': datetime.datetime(2017, 6, 30, 0, 0),
 '2017-q3': datetime.datetime(2017, 9, 30, 0, 0),
 '2017-q4': datetime.datetime(2017, 12, 31, 0, 0),
 '2018-q1': datetime.datetime(2018, 3, 31, 0, 0),
 '2018-q2': datetime.datetime(2018, 6, 30, 0, 0),
 '2018-q3': datetime.datetime(2018, 9, 30, 0, 0),
 '2018-q4': datetime.datetime(2018, 12, 31, 0, 0),
 '2019-q1': datetime.datetime(2019, 3, 31, 0, 0),
 '2019-q2': datetime.datetime(2019, 6, 30, 0, 0),
 '2019-q3': datetime.datetime(2019, 9, 30, 0, 0),
 '2019-q4': datetime.datetime(2019, 12, 31, 0, 0),
 '2020-q1': datetime.datetime(2020, 3, 31, 0, 0),
 '2020-q2': datetime.datetime(2020, 6, 30, 0, 0)}

In [63]:
for area in stt_count:
    # [total_num_station, total_duration, avg_duration]
    stt_count[area] = {q: [0, 0, 0] for q in quarter_end_date}

In [64]:
stt_count

{'DTLA': {'2016-q3': [0, 0, 0],
  '2016-q4': [0, 0, 0],
  '2017-q1': [0, 0, 0],
  '2017-q2': [0, 0, 0],
  '2017-q3': [0, 0, 0],
  '2017-q4': [0, 0, 0],
  '2018-q1': [0, 0, 0],
  '2018-q2': [0, 0, 0],
  '2018-q3': [0, 0, 0],
  '2018-q4': [0, 0, 0],
  '2019-q1': [0, 0, 0],
  '2019-q2': [0, 0, 0],
  '2019-q3': [0, 0, 0],
  '2019-q4': [0, 0, 0],
  '2020-q1': [0, 0, 0],
  '2020-q2': [0, 0, 0]},
 'Port of LA': {'2016-q3': [0, 0, 0],
  '2016-q4': [0, 0, 0],
  '2017-q1': [0, 0, 0],
  '2017-q2': [0, 0, 0],
  '2017-q3': [0, 0, 0],
  '2017-q4': [0, 0, 0],
  '2018-q1': [0, 0, 0],
  '2018-q2': [0, 0, 0],
  '2018-q3': [0, 0, 0],
  '2018-q4': [0, 0, 0],
  '2019-q1': [0, 0, 0],
  '2019-q2': [0, 0, 0],
  '2019-q3': [0, 0, 0],
  '2019-q4': [0, 0, 0],
  '2020-q1': [0, 0, 0],
  '2020-q2': [0, 0, 0]},
 'Pasadena': {'2016-q3': [0, 0, 0],
  '2016-q4': [0, 0, 0],
  '2017-q1': [0, 0, 0],
  '2017-q2': [0, 0, 0],
  '2017-q3': [0, 0, 0],
  '2017-q4': [0, 0, 0],
  '2018-q1': [0, 0, 0],
  '2018-q2': [0, 0, 0],
  '2

## number of stations

In [65]:
for area in stt_count:
    df_area = df_get[df_get['region'] == area]
    for q in quarter_end_date:
        stt_c = df_area[df_area.go_live_date <= quarter_end_date[q]].shape[0]
        stt_count[area][q][0] = stt_c

In [66]:
stt_count

{'DTLA': {'2016-q3': [62, 0, 0],
  '2016-q4': [62, 0, 0],
  '2017-q1': [62, 0, 0],
  '2017-q2': [62, 0, 0],
  '2017-q3': [64, 0, 0],
  '2017-q4': [65, 0, 0],
  '2018-q1': [65, 0, 0],
  '2018-q2': [68, 0, 0],
  '2018-q3': [73, 0, 0],
  '2018-q4': [77, 0, 0],
  '2019-q1': [86, 0, 0],
  '2019-q2': [97, 0, 0],
  '2019-q3': [111, 0, 0],
  '2019-q4': [130, 0, 0],
  '2020-q1': [138, 0, 0],
  '2020-q2': [141, 0, 0]},
 'Port of LA': {'2016-q3': [0, 0, 0],
  '2016-q4': [0, 0, 0],
  '2017-q1': [0, 0, 0],
  '2017-q2': [0, 0, 0],
  '2017-q3': [12, 0, 0],
  '2017-q4': [12, 0, 0],
  '2018-q1': [12, 0, 0],
  '2018-q2': [12, 0, 0],
  '2018-q3': [12, 0, 0],
  '2018-q4': [12, 0, 0],
  '2019-q1': [12, 0, 0],
  '2019-q2': [12, 0, 0],
  '2019-q3': [13, 0, 0],
  '2019-q4': [13, 0, 0],
  '2020-q1': [13, 0, 0],
  '2020-q2': [13, 0, 0]},
 'Pasadena': {'2016-q3': [0, 0, 0],
  '2016-q4': [0, 0, 0],
  '2017-q1': [0, 0, 0],
  '2017-q2': [0, 0, 0],
  '2017-q3': [31, 0, 0],
  '2017-q4': [31, 0, 0],
  '2018-q1': [31, 

## total duration

In [112]:
att_collection = [list(df_get[df_get.region == area].station) + [area] for area in df_get.region.unique()]

In [113]:
for q in quarter_end_date:
    pth_trip = '../processed_data/metro-bike-' + q + '.csv'
    print(pth_trip)
    df = pd.read_csv(pth_trip)
    for row in df[['start_station', 'duration']].iterrows():
        stt_id = row[1]['start_station']
        dur = row[1]['duration']
        for att in att_collection:
            if stt_id in att:
                area_belong = att[-1]
                stt_count[area_belong][q][1] += dur
                continue

../processed_data/metro-bike-2016-q3.csv
../processed_data/metro-bike-2016-q4.csv
../processed_data/metro-bike-2017-q1.csv
../processed_data/metro-bike-2017-q2.csv
../processed_data/metro-bike-2017-q3.csv
../processed_data/metro-bike-2017-q4.csv
../processed_data/metro-bike-2018-q1.csv
../processed_data/metro-bike-2018-q2.csv
../processed_data/metro-bike-2018-q3.csv
../processed_data/metro-bike-2018-q4.csv
../processed_data/metro-bike-2019-q1.csv
../processed_data/metro-bike-2019-q2.csv
../processed_data/metro-bike-2019-q3.csv
../processed_data/metro-bike-2019-q4.csv
../processed_data/metro-bike-2020-q1.csv
../processed_data/metro-bike-2020-q2.csv


In [114]:
stt_count

{'DTLA': {'2016-q3': [62, 1326127, 0],
  '2016-q4': [62, 900473, 0],
  '2017-q1': [62, 718001, 0],
  '2017-q2': [62, 971828, 0],
  '2017-q3': [64, 1070883, 0],
  '2017-q4': [65, 994306, 0],
  '2018-q1': [65, 827460, 0],
  '2018-q2': [68, 1024520, 0],
  '2018-q3': [73, 1747080, 0],
  '2018-q4': [77, 1631589, 0],
  '2019-q1': [86, 1124141, 0],
  '2019-q2': [97, 1125137, 0],
  '2019-q3': [111, 1672195, 0],
  '2019-q4': [130, 1336158, 0],
  '2020-q1': [138, 1238003, 0],
  '2020-q2': [141, 379163, 0]},
 'Port of LA': {'2016-q3': [0, 0, 0],
  '2016-q4': [0, 0, 0],
  '2017-q1': [0, 0, 0],
  '2017-q2': [0, 0, 0],
  '2017-q3': [12, 106159, 0],
  '2017-q4': [12, 65369, 0],
  '2018-q1': [12, 61109, 0],
  '2018-q2': [12, 83014, 0],
  '2018-q3': [12, 175646, 0],
  '2018-q4': [12, 58061, 0],
  '2019-q1': [12, 55725, 0],
  '2019-q2': [12, 138168, 0],
  '2019-q3': [13, 172760, 0],
  '2019-q4': [13, 21503, 0],
  '2020-q1': [13, 0, 0],
  '2020-q2': [13, 0, 0]},
 'Pasadena': {'2016-q3': [0, 0, 0],
  '201

In [116]:
for area in stt_count:
    for q in stt_count[area]:
        if stt_count[area][q][0] == 0:
            stt_count[area][q][1] = 0

In [117]:
stt_count

{'DTLA': {'2016-q3': [62, 1326127, 0],
  '2016-q4': [62, 900473, 0],
  '2017-q1': [62, 718001, 0],
  '2017-q2': [62, 971828, 0],
  '2017-q3': [64, 1070883, 0],
  '2017-q4': [65, 994306, 0],
  '2018-q1': [65, 827460, 0],
  '2018-q2': [68, 1024520, 0],
  '2018-q3': [73, 1747080, 0],
  '2018-q4': [77, 1631589, 0],
  '2019-q1': [86, 1124141, 0],
  '2019-q2': [97, 1125137, 0],
  '2019-q3': [111, 1672195, 0],
  '2019-q4': [130, 1336158, 0],
  '2020-q1': [138, 1238003, 0],
  '2020-q2': [141, 379163, 0]},
 'Port of LA': {'2016-q3': [0, 0, 0],
  '2016-q4': [0, 0, 0],
  '2017-q1': [0, 0, 0],
  '2017-q2': [0, 0, 0],
  '2017-q3': [12, 106159, 0],
  '2017-q4': [12, 65369, 0],
  '2018-q1': [12, 61109, 0],
  '2018-q2': [12, 83014, 0],
  '2018-q3': [12, 175646, 0],
  '2018-q4': [12, 58061, 0],
  '2019-q1': [12, 55725, 0],
  '2019-q2': [12, 138168, 0],
  '2019-q3': [13, 172760, 0],
  '2019-q4': [13, 21503, 0],
  '2020-q1': [13, 0, 0],
  '2020-q2': [13, 0, 0]},
 'Pasadena': {'2016-q3': [0, 0, 0],
  '201

## avg duration time

In [118]:
for area in stt_count:
    for q in stt_count[area]:
        if stt_count[area][q][0] != 0:
            stt_count[area][q][2] = stt_count[area][q][1] / stt_count[area][q][0]
        else:
            stt_count[area][q][2] = 0

In [119]:
stt_count

{'DTLA': {'2016-q3': [62, 1326127, 21389.145161290322],
  '2016-q4': [62, 900473, 14523.758064516129],
  '2017-q1': [62, 718001, 11580.661290322581],
  '2017-q2': [62, 971828, 15674.645161290322],
  '2017-q3': [64, 1070883, 16732.546875],
  '2017-q4': [65, 994306, 15297.015384615384],
  '2018-q1': [65, 827460, 12730.153846153846],
  '2018-q2': [68, 1024520, 15066.470588235294],
  '2018-q3': [73, 1747080, 23932.602739726026],
  '2018-q4': [77, 1631589, 21189.467532467534],
  '2019-q1': [86, 1124141, 13071.406976744185],
  '2019-q2': [97, 1125137, 11599.350515463917],
  '2019-q3': [111, 1672195, 15064.81981981982],
  '2019-q4': [130, 1336158, 10278.138461538461],
  '2020-q1': [138, 1238003, 8971.036231884058],
  '2020-q2': [141, 379163, 2689.099290780142]},
 'Port of LA': {'2016-q3': [0, 0, 0],
  '2016-q4': [0, 0, 0],
  '2017-q1': [0, 0, 0],
  '2017-q2': [0, 0, 0],
  '2017-q3': [12, 106159, 8846.583333333334],
  '2017-q4': [12, 65369, 5447.416666666667],
  '2018-q1': [12, 61109, 5092.416

## if duration == 0, station == 0

In [120]:
for area in stt_count:
    for q in stt_count[area]:
        if stt_count[area][q][1] == 0:
            stt_count[area][q][0] = 0

In [121]:
stt_count

{'DTLA': {'2016-q3': [62, 1326127, 21389.145161290322],
  '2016-q4': [62, 900473, 14523.758064516129],
  '2017-q1': [62, 718001, 11580.661290322581],
  '2017-q2': [62, 971828, 15674.645161290322],
  '2017-q3': [64, 1070883, 16732.546875],
  '2017-q4': [65, 994306, 15297.015384615384],
  '2018-q1': [65, 827460, 12730.153846153846],
  '2018-q2': [68, 1024520, 15066.470588235294],
  '2018-q3': [73, 1747080, 23932.602739726026],
  '2018-q4': [77, 1631589, 21189.467532467534],
  '2019-q1': [86, 1124141, 13071.406976744185],
  '2019-q2': [97, 1125137, 11599.350515463917],
  '2019-q3': [111, 1672195, 15064.81981981982],
  '2019-q4': [130, 1336158, 10278.138461538461],
  '2020-q1': [138, 1238003, 8971.036231884058],
  '2020-q2': [141, 379163, 2689.099290780142]},
 'Port of LA': {'2016-q3': [0, 0, 0],
  '2016-q4': [0, 0, 0],
  '2017-q1': [0, 0, 0],
  '2017-q2': [0, 0, 0],
  '2017-q3': [12, 106159, 8846.583333333334],
  '2017-q4': [12, 65369, 5447.416666666667],
  '2018-q1': [12, 61109, 5092.416

## save

In [124]:
for area in stt_count:
    for q in stt_count[area]:
        stt_count[area][q][0] = int(stt_count[area][q][0])
        stt_count[area][q][1] = int(stt_count[area][q][1])
        stt_count[area][q][2] = float(stt_count[area][q][2])

In [125]:
with open('../chart_data/overview/station_overview.json', 'w', encoding='utf-8') as fp:
    json.dump(stt_count, fp)